In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
173,ABW,North America,Aruba,2020-09-07,2449.0,21.0,64.571,14.0,0.0,0.571,...,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29
425,AFG,Asia,Afghanistan,2020-09-07,38398.0,74.0,33.714,1412.0,2.0,1.429,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
595,AGO,Africa,Angola,2020-09-07,2935.0,59.0,44.429,117.0,2.0,1.429,...,1.362,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN,61.15


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-06,France,325284.0,0.0,30719.0,0.0,0.0,0.0
0,2020-09-06,China,90049.0,0.0,4734.0,0.0,0.0,0.0
0,2020-09-06,Italy,277846.0,0.0,35563.0,0.0,0.0,0.0
0,2020-09-06,Spain,505623.0,0.0,29560.0,0.0,0.0,0.0
0,2020-09-06,United States,6325744.0,0.0,189723.0,0.0,0.0,0.0
0,2020-09-06,World,27381941.0,0.0,889412.0,0.0,0.0,0.0
0,2020-09-06,United Kingdom,345947.0,0.0,41592.0,0.0,0.0,0.0
0,2020-09-06,Germany,251796.0,0.0,9335.0,0.0,0.0,0.0
0,2020-09-06,Iran,388667.0,0.0,22401.0,0.0,0.0,0.0
0,2020-09-06,Turkey,279527.0,0.0,6662.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-07,France,325284.0,324777.0,30719.0,30701.0,507.0,18.0
0,2020-09-07,China,90049.0,90058.0,4734.0,4730.0,-9.0,4.0
0,2020-09-07,Italy,277846.0,277634.0,35563.0,35541.0,212.0,22.0
0,2020-09-06,Spain,505623.0,498989.0,29560.0,29418.0,6634.0,142.0
0,2020-09-07,United States,6325744.0,6276421.0,189723.0,188941.0,49323.0,782.0
0,2020-09-07,World,27381941.0,27150797.0,889412.0,889256.0,231144.0,156.0
0,2020-09-07,United Kingdom,345947.0,347152.0,41592.0,41551.0,-1205.0,41.0
0,2020-09-07,Germany,251796.0,250799.0,9335.0,9325.0,997.0,10.0
0,2020-09-07,Iran,388667.0,386658.0,22401.0,22293.0,2009.0,108.0
0,2020-09-07,Turkey,279527.0,279806.0,6662.0,6673.0,-279.0,-11.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")